In [1]:
import os
import re
import pandas as pd
import urllib
from nltk import SnowballStemmer
from gensim.models import Word2Vec
#import langdetect
import tika
import time
from tika import parser
import pickle

# Raw Articles pdf Data
Convert pdf to text using Tika apache server 

In [2]:
# specify the target data folder-
target_dir = 'C:/Thesis/Data/Human_Resource_Management'

In [3]:
# prep pdf extraction
pdf_files = []
wrd_files = []

for f in os.listdir(target_dir):
    if f.endswith(".pdf") | f.endswith(".PDF"):
        thispdf = os.path.join(target_dir, f)
        pdf_files = pdf_files + [thispdf]
    if f.endswith(".doc") | f.endswith(".docx") | f.endswith(".DOC") | f.endswith(".DOCX"):
        thiswrd = os.path.join(target_dir, f)
        wrd_files = wrd_files + [thiswrd]

In [4]:
port = 4321 # port to use for Tika server (chosen arbitrarily)
server_str = 'http://localhost:' + str(port)
tika_path  = 'C:\Software\ tika-server-1.23.jar'
tika_run   = 'java -jar' + tika_path + ' --port ' + str(port)
print('Command to start Tika:')
print(tika_run)
print('---')
print('Number of pdf files: ' + str(len(pdf_files)))
print('Number of word files: ' + str(len(wrd_files)))

Command to start Tika:
java -jarC:\Software\ tika-server-1.23.jar --port 4321
---
Number of pdf files: 903
Number of word files: 0


# pdf metadata

In [6]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

pdf_metadata = [parser.from_file(t, server_str, xmlContent=False)["metadata"] for t in (pdf_files + wrd_files)]

In [42]:
# save data metadata
pickle.dump(pdf_metadata, open( "C:/Thesis/Data/save/Human_Resource_Management/metadata/metadata_all.p", "wb" ) )

# pdf content

In [ ]:
# process the PDF files, store text in memory
tika.TikaClientOnly = True 

#pdf_text = [parser.from_file(t, server_str, xmlContent=False)["content"] for t in (pdf_files + wrd_files)]

# step-1 load pdf data

In [2]:
import pickle
#load the text data
pdf_text = pickle.load(open("C:/Thesis/Data/save/Human_Resource_Management/save_alltextData.p", "rb"))


# step-2 convert it to lowercase

In [3]:
#Convert each string to lowercase
for i in range(len(pdf_text)):
    pdf_text[i] = pdf_text[i].lower()

# step-3 remove reference

In [4]:
# remove references section 
import re
pdf_ref = [re.sub(r"(?is)\nreferences\n.+", "", f) for f in pdf_text]

# step-4 load metadata

In [1]:
import pickle
#load the metadata
pdf_metadata = pickle.load(open("C:/Thesis/Data/save/Human_Resource_Management/metadata/metadata_all.p", "rb"))

# step-5 get metadata = author, subject, title

In [5]:
pdf_title = []
pdf_Author = []
pdf_subject = []


for i in pdf_metadata:
    temp =""
    auth_temp = ""
    subj_temp = ""
    try:
        temp = i['title']
        auth_temp = i['Author']
        subj_temp = i['subject']
     
        pdf_title.append(temp.lower()) 
        pdf_Author.append(auth_temp.lower()) 
        pdf_subject.append(subj_temp.lower())
       
    except:
        #print("An exception occurred" )
        pdf_title.append(temp.lower())
        pdf_Author.append(auth_temp.lower())
        pdf_subject.append(subj_temp.lower())
       

In [6]:
#pdf_title
pdf_subject[316]


'human resource management 2016.55:301-320'

In [50]:
# pre-process author
import re
pdf_auth = []

for j in pdf_Author:
    k = re.sub(' and ', ',', j)
    k = re.sub(' & ', ',', k)
    k = k.split(",")
    pdf_auth.append(k)
    

In [ ]:
# remove author from the content 
pdf_text_author = []
for i in pdf_data:
    line = i
    # for author
    for j in pdf_auth:
        for k in j:
            line = re.sub(k.strip(), '', line)
        
    pdf_text_author.append(line)

In [57]:
# save removed author
pickle.dump(pdf_text_author, open( "C:/Thesis/Data/save/Human_Resource_Management/metadata/removed_author/metadata_all.p", "wb" ) )


In [58]:
# remove title
pdf_text_title=[]
for j in pdf_text_author:
    line = j
    for k in pdf_title:
        line = re.sub(k.strip(), '', line)
        
    pdf_text_title.append(line)

In [60]:
# save removed title
pickle.dump(pdf_text_title, open( "C:/Thesis/Data/save/Human_Resource_Management/metadata/removed_author_title/metadata_all.p", "wb" ) )


In [67]:
# preprocess the subject
#Human Resource Management 2016.55:301-320

pdf_sub = []

for i in pdf_subject:
    # remove special character
    t = (re.sub(r'[?\-|.|:|!]','',i))
    # remove digits
    t = (re.sub("\d+", "", t))
    # remove end whitespace
    t = re.sub(r"\s+$","",t)
    # append to the list
    pdf_sub.append(t)



In [69]:
# remove subject from above list
pdf_text_subject = []
for j in pdf_text_title:
    line = j
    for k in pdf_sub:
        line = re.sub(k.strip(), '', line)
    pdf_text_subject.append(line)

In [120]:
#pdf_text_subject[0]

In [70]:
# save removed title
pickle.dump(pdf_text_subject, open( "C:/Thesis/Data/save/Human_Resource_Management/metadata/removed_author_title_sub/metadata_all.p", "wb" ) )

In [8]:
import pickle
#load the data
pdf_text_subject = pickle.load(open("C:/Thesis/Data/save/Human_Resource_Management/metadata/removed_author_title_sub/metadata_all.p", "rb"))

In [109]:
pdf_text_subject[10]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nthe relationship between line manager behavior, perceived hrm practices, and individual performance: examining the mediating role of engagement\n\n\nthe relationship between \n\nline manager behavior, \n\nperceived hrm practices, and \n\nindividual performance: \n\nexamining the mediating \n\nrole of engagement\n\nk e r s t i n  a l f e s ,  c at h e r i n e  t r u s s ,  e m m a  c . \ns o a n e ,  c h r i s  r e e s ,  a n d  m a r k  g at e n b y\n\nthis article examines the role played by line managers in the link between \n\nhrm practices and individual performance outcomes. drawing on social ex-\n\nchange theory, the authors test a mediated model linking perceived line man-\n\nager behavior and perceived  practices with \n\nemployee engagement and individual performance. the study focuses on \n\ntwo self-report measures of individual performance; task performance and \n\ninnovative work b

In [ ]:
# not required
def eliminate_method_result(method, discussion):
    
    import re    
    start = method
    end = discussion
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]

    return pdf_method_dis

In [ ]:
# not required
def eliminate_method_result_ref(method, dis, ref):
    
    import re

    start = method
    end = dis
    
    #start of pattern, followed by any number of times 'any character OR a newline' and terminated by the end pattern.
    pattern = start + '.*'+'(.|\n)*'+end
    pdf_method_dis = [re.sub(pattern, '', f) for f in pdf_text]
    
    start_ref = ref
    pattern_ref = start_ref +'(.|\n)*$'

    pdf_ref= [re.sub(pattern_ref, '', f) for f in pdf_method_dis]
    
    return pdf_ref

# Pre-processing
Regular expression is used to clean up the pre-processed data and saved as a Master data

In [94]:
pdf_p= [re.sub(r"-\n(\n)*", "", t)for t in pdf_text_subject]

pdf_p= [re.sub(r"\n(\n)*", "\n", t)for t in pdf_p]

pdf_p= [re.sub(r"\n", " ", t)for t in pdf_p]
# remove text with [] eg citation
pdf_p = [re.sub(r"\[[^)]*\]", "", t)for t in pdf_p]
#Remove numbers
pdf_p = [re.sub('[0-9]+', '', f) for f in pdf_p]
#pdf_p = [re.sub("r[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?", " ", t)for t in pdf_p]
pdf_p = [re.sub('[!@/%“”‘:#©β<>+=δχ*&$]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('’s','', f) for f in pdf_p]
pdf_p = [re.sub('’re','', f) for f in pdf_p]
pdf_p = [re.sub('’t','', f) for f in pdf_p]
pdf_p = [re.sub('’ve','', f) for f in pdf_p]
pdf_p = [re.sub('’ll','', f) for f in pdf_p]
pdf_p = [re.sub('’m','', f) for f in pdf_p]

pdf_p = [re.sub('[:()-]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('[\.!?]+ ', 'XYZXYZ', t) for t in pdf_p]
# \w will match alphanumeric characters and underscores
# [^\w] will match anything that's not alphanumeric or underscore
pdf_p = [re.sub(r'[^\w]', ' ', f) for f in pdf_p]
pdf_p = [re.sub('XYZXYZ', '.', f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]
# Replace multiple dots with one dot
pdf_p = [re.sub('\.\.+',".", f) for f in pdf_p]
pdf_p = [re.sub(r'\b\w{1,3}\b',"", f) for f in pdf_p]
pdf_p = [re.sub(' +', ' ', f) for f in pdf_p]

pdf_p = [re.sub("^\s+","", f) for f in pdf_p]# remove the front space
pdf_p = [re.sub("\s+\Z","", f) for f in pdf_p]#remove the back space

In [95]:
pdf_p[10]

'relationship between line manager behavior perceived practices individual performance examining mediating role engagement relationship between line manager behavior perceived practices individual performance examining mediating role engagement . this article examines role played line managers link between practices individual performance outcomes.drawing social exchange theory authors test mediated model linking perceived line manager behavior perceived practices with employee engagement individual performance. study focuses self report measures individual performance task performance innovative work behavior. studies with total participants were conducted service sector organizations united kingdom analyzed using structural equation modeling. data reveal that perceived line manager behavior perceived practices linked with employee engagement. turn employee engagement strongly linked individual performance fully mediates link between both perceived practices perceived line manager beh

In [96]:
stopwords = ['a', 'about', 'above', 'across', 'after', 'afterwards']
stopwords += ['again', 'against', 'all', 'almost', 'alone', 'along']
stopwords += ['already', 'also', 'although', 'always', 'am', 'among']
stopwords += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stopwords += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stopwords += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stopwords += ['because', 'become', 'becomes', 'becoming', 'been']
stopwords += ['before', 'beforehand', 'behind', 'being', 'below']
stopwords += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stopwords += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stopwords += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stopwords += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stopwords += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stopwords += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stopwords += ['every', 'everyone', 'everything', 'everywhere', 'except']
stopwords += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stopwords += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stopwords += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stopwords += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stopwords += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stopwords += ['herself', 'him', 'himself', 'his', 'how', 'however']
stopwords += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stopwords += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stopwords += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stopwords += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stopwords += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stopwords += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stopwords += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stopwords += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stopwords += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stopwords += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stopwords += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stopwords += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stopwords += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stopwords += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stopwords += ['some', 'somehow', 'someone', 'something', 'sometime']
stopwords += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stopwords += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stopwords += ['then', 'thence', 'there', 'thereafter', 'thereby']
stopwords += ['therefore', 'therein', 'thereupon', 'these', 'they']
stopwords += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stopwords += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stopwords += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stopwords += ['twenty', 'two', 'un', 'under', 'until', 'up', 'upon']
stopwords += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stopwords += ['whatever', 'when', 'whence', 'whenever', 'where']
stopwords += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stopwords += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stopwords += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stopwords += ['within', 'without', 'would', 'yet', 'you', 'your']
stopwords += ['yours', 'yourself', 'yourselves']
stopwords += ['january', 'february', 'march', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
# author name to remove
stopwords +=['karina nielsen','kevin daniels','elaine', 'jaewan yang','andrew O herdman','amanda', 'sabine','kerstin','kertin','sagepub','journalspermissions','catheine','chidiebere ogbonnaya','john wiley','sons ltd','martin','jeffrey B arthur']

In [97]:
# Function to filter out the stopwords and authors names
def filterWords(msg, words):
    m = msg.split(' ')
    words_end = [w + '.' for w in words]
    filtered_words = list(filter(lambda word: word not in words, m))
    filtered_words = [(w if w not in words_end else '.') for w in filtered_words]
    result = ' '.join(filtered_words)
    return(result)

In [99]:
data_proc = [filterWords(f, stopwords) for f in pdf_p]

# Get country name

In [130]:
import pycountry

country_list = []
for country in pycountry.countries:
    a = (country.name).lower()
    country_list.append(a) 

In [131]:
data_proc_country =[]
for j in data_proc:
    line = j
    for k in country_list:
        line = re.sub(k, '', line)
        
    data_proc_country.append(line)

In [135]:
#data_proc_country[10]

In [136]:
# save removed title
pickle.dump(data_proc_country, open( "C:/Thesis/Data/save/Human_Resource_Management/metadata/removed_country/metadata_all.p", "wb" ) )

In [137]:
# Authors names are saved already (code is available at author name folder-get name list notebook)
#load authors names
import pickle
author_name = pickle.load(open("C:/Thesis/Data/save/Master_Data/auth_ref_filter_num_char/auth_Human_Resource_Management.p", "rb"))

In [138]:
from nltk.corpus import stopwords

sw = stopwords.words('english') + author_name 

In [140]:
data_proc_1 = [filterWords(f, sw) for f in data_proc_country]

In [141]:
data_proc_1[10]

'relationship line manager behavior perceived practices individual examining mediating role engagement relationship line manager behavior perceived practices individual examining mediating role engagement . examines role played line managers practices individual outcomes.drawing theory authors test mediated linking perceived line manager behavior perceived practices employee engagement individual . focuses measures individual task innovative behavior. total participants conducted sector organizations  analyzed using structural equation modeling. reveal perceived line manager behavior perceived practices linked employee engagement. turn employee engagement strongly linked individual fully mediates perceived practices perceived line manager behavior task innovative behavior . ndings signifi cance line manager mediating role played employee engagement. periodicals .keywords perceived practices perceived line manager behavior employee engagement task innovative behavior correspondence tilb

In [185]:
data_proc_2 = [re.sub(' \.+', ' ', f) for f in data_proc_1]
data_proc_2 = [re.sub(' +', ' ', f) for f in data_proc_2]
data_proc_2 = [re.sub('http', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('www', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('iweb', '', f) for f in data_proc_2]
data_proc_2 = [re.sub('ttimmerman ', ' ', f) for f in data_proc_2]
data_proc_2 = [re.sub(' +', ' ', f) for f in data_proc_2]


In [187]:
# save data
pickle.dump(data_proc_2, open( "C:/Thesis/Data/save/Master_Data/MD_1/data_proc_latest/HRM_data_proc.p", "wb" ) )

In [188]:
#import io
with open("C:/Thesis/Data/save/Master_Data/MD_1/data_proc_latest/HRM_data_proc_txt.txt", "w",encoding="utf-8") as outfile:
    for i in range(len(data_proc_2)):
        outstring = ""
        outstring += str(data_proc_2[i])
        outfile.write(outstring)